# Table Preparation

This notebook prepares some tables.

In [54]:
import pandas as pd
import numpy as np

## Calibration Table

We want that big calibration table.

In [2]:
calib = pd.read_csv('calibration_table.tsv', sep='\t')
calib.head()

Data       Opt.     Model  I-I Sim  Item Pop  U-U Sim  User Act
0  ML1M  Avg. Loss  IBP-Unif    0.170     0.973    0.227     0.373
1  ML1M  Avg. Loss   IBP-Pop    0.200     1.793    0.170     0.388
2  ML1M  Avg. Loss  LDA-Unif    0.137     0.737    0.082     0.669
3  ML1M  Avg. Loss   LDA-Pop    0.050     2.063    0.291     1.084
4  ML1M    I-I Sim  IBP-Unif    0.218     0.846    0.185     0.477

In [4]:
calib_avg = calib[calib['Opt.'] == 'Avg. Loss'].drop(columns=['Opt.']).set_index(['Data', 'Model'])
calib_avg.head()

I-I Sim  Item Pop  U-U Sim  User Act
Data Model                                         
ML1M IBP-Unif    0.170     0.973    0.227     0.373
     IBP-Pop     0.200     1.793    0.170     0.388
     LDA-Unif    0.137     0.737    0.082     0.669
     LDA-Pop     0.050     2.063    0.291     1.084
AZM5 IBP-Unif    0.024     4.639    0.020     0.100

In [13]:
calib_ref = calib[calib['Opt.'] != 'Avg. Loss'].reset_index(drop=True)
calib_ref.head()

Data      Opt.     Model  I-I Sim  Item Pop  U-U Sim  User Act
0  ML1M   I-I Sim  IBP-Unif    0.218     0.846    0.185     0.477
1  ML1M   I-I Sim   IBP-Pop    0.081     1.184    0.294     0.361
2  ML1M   I-I Sim  LDA-Unif    0.011     0.460    0.355     0.397
3  ML1M   I-I Sim   LDA-Pop    0.002     0.490    0.163     0.478
4  ML1M  Item Pop  IBP-Unif    1.385     0.791    1.132     4.022

In [17]:
def best(df):
    col = df.name[-1]
    best = df.nsmallest(1, col)
    return best[['Model', col]].rename(columns={col: 'Divergence'})

calib_rb = calib_ref.groupby(['Data', 'Opt.']).apply(best)
calib_rb.reset_index(level=-1, drop=True, inplace=True)
calib_rb

Model  Divergence
Data  Opt.                          
AZM5  I-I Sim    LDA-Pop       0.004
      Item Pop   LDA-Pop       0.419
      U-U Sim   LDA-Unif       0.001
      User Act  IBP-Unif       0.033
ML1M  I-I Sim    LDA-Pop       0.002
      Item Pop   LDA-Pop       0.458
      U-U Sim   LDA-Unif       0.006
      User Act  LDA-Unif       0.262
STMV1 I-I Sim   IBP-Unif       0.140
      Item Pop  IBP-Unif       0.373
      U-U Sim    LDA-Pop       0.077
      User Act  LDA-Unif       0.353

In [24]:
cref = calib_rb.reset_index().pivot(columns='Opt.', values='Divergence', index='Data')
cref

Opt.   I-I Sim  Item Pop  U-U Sim  User Act
Data                                       
AZM5     0.004     0.419    0.001     0.033
ML1M     0.002     0.458    0.006     0.262
STMV1    0.140     0.373    0.077     0.353

In [29]:
rel_loss = (calib_avg - cref) / cref
rel_loss

I-I Sim   Item Pop    U-U Sim   User Act
Data  Model                                               
ML1M  IBP-Unif  84.000000   1.124454  36.833333   0.423664
      IBP-Pop   99.000000   2.914847  27.333333   0.480916
      LDA-Unif  67.500000   0.609170  12.666667   1.553435
      LDA-Pop   24.000000   3.504367  47.500000   3.137405
AZM5  IBP-Unif   5.000000  10.071599  19.000000   2.030303
      IBP-Pop    3.000000   8.952267  19.000000   3.363636
      LDA-Unif   2.750000   1.455847   1.000000   2.454545
      LDA-Pop    7.750000   0.157518   7.000000   0.363636
STMV1 IBP-Unif  14.021429   2.482574  21.324675  11.756374
      IBP-Pop    1.614286   5.589812   9.000000   6.042493
      LDA-Unif  33.878571   6.067024  77.597403   7.781870
      LDA-Pop   33.828571   5.517426  77.454545   8.209632

In [41]:
fit_merge = pd.concat({'KLD': calib_avg, 'Loss': rel_loss}, axis=1)
fit_merge = fit_merge.reorder_levels([1,0], 1).sort_index(1)
fit_merge

I-I Sim            Item Pop            U-U Sim             \
                   KLD       Loss      KLD       Loss     KLD       Loss   
Data  Model                                                                
ML1M  IBP-Unif   0.170  84.000000    0.973   1.124454   0.227  36.833333   
      IBP-Pop    0.200  99.000000    1.793   2.914847   0.170  27.333333   
      LDA-Unif   0.137  67.500000    0.737   0.609170   0.082  12.666667   
      LDA-Pop    0.050  24.000000    2.063   3.504367   0.291  47.500000   
AZM5  IBP-Unif   0.024   5.000000    4.639  10.071599   0.020  19.000000   
      IBP-Pop    0.016   3.000000    4.170   8.952267   0.020  19.000000   
      LDA-Unif   0.015   2.750000    1.029   1.455847   0.002   1.000000   
      LDA-Pop    0.035   7.750000    0.485   0.157518   0.008   7.000000   
STMV1 IBP-Unif   2.103  14.021429    1.299   2.482574   1.719  21.324675   
      IBP-Pop    0.366   1.614286    2.458   5.589812   0.770   9.000000   
      LDA-Unif   4.883  33.878571    2.636   6.067024   6.052  77.597403   
      LDA-Pop    4.876  33.828571    2.431   5.517426   6.041  77.454545   

               User Act             
                    KLD       Loss  
Data  Model                         
ML1M  IBP-Unif    0.373   0.423664  
      IBP-Pop     0.388   0.480916  
      LDA-Unif    0.669   1.553435  
      LDA-Pop     1.084   3.137405  
AZM5  IBP-Unif    0.100   2.030303  
      IBP-Pop     0.144   3.363636  
      LDA-Unif    0.114   2.454545  
      LDA-Pop     0.045   0.363636  
STMV1 IBP-Unif    4.503  11.756374  
      IBP-Pop     2.486   6.042493  
      LDA-Unif    3.100   7.781870  
      LDA-Pop     3.251   8.209632

In [60]:
metrics = fit_merge.columns.levels[0]
losses = fit_merge.reorder_levels([1,0], 1).loc[:, 'Loss']
losses.mean(axis=1)

Data   Model   
ML1M   IBP-Unif    30.595363
       IBP-Pop     32.432274
       LDA-Unif    20.582318
       LDA-Pop     19.535443
AZM5   IBP-Unif     9.025476
       IBP-Pop      8.578976
       LDA-Unif     1.915098
       LDA-Pop      3.817789
STMV1  IBP-Unif    12.396263
       IBP-Pop      5.561648
       LDA-Unif    31.331217
       LDA-Pop     31.252544
dtype: float64

In [61]:
fit_merge[('Avg', 'Loss')] = losses.mean(axis=1)
fit_merge

I-I Sim            Item Pop            U-U Sim             \
                   KLD       Loss      KLD       Loss     KLD       Loss   
Data  Model                                                                
ML1M  IBP-Unif   0.170  84.000000    0.973   1.124454   0.227  36.833333   
      IBP-Pop    0.200  99.000000    1.793   2.914847   0.170  27.333333   
      LDA-Unif   0.137  67.500000    0.737   0.609170   0.082  12.666667   
      LDA-Pop    0.050  24.000000    2.063   3.504367   0.291  47.500000   
AZM5  IBP-Unif   0.024   5.000000    4.639  10.071599   0.020  19.000000   
      IBP-Pop    0.016   3.000000    4.170   8.952267   0.020  19.000000   
      LDA-Unif   0.015   2.750000    1.029   1.455847   0.002   1.000000   
      LDA-Pop    0.035   7.750000    0.485   0.157518   0.008   7.000000   
STMV1 IBP-Unif   2.103  14.021429    1.299   2.482574   1.719  21.324675   
      IBP-Pop    0.366   1.614286    2.458   5.589812   0.770   9.000000   
      LDA-Unif   4.883  33.878571    2.636   6.067024   6.052  77.597403   
      LDA-Pop    4.876  33.828571    2.431   5.517426   6.041  77.454545   

               User Act                   Avg  
                    KLD       Loss       Loss  
Data  Model                                    
ML1M  IBP-Unif    0.373   0.423664  30.595363  
      IBP-Pop     0.388   0.480916  32.432274  
      LDA-Unif    0.669   1.553435  20.582318  
      LDA-Pop     1.084   3.137405  19.535443  
AZM5  IBP-Unif    0.100   2.030303   9.025476  
      IBP-Pop     0.144   3.363636   8.578976  
      LDA-Unif    0.114   2.454545   1.915098  
      LDA-Pop     0.045   0.363636   3.817789  
STMV1 IBP-Unif    4.503  11.756374  12.396263  
      IBP-Pop     2.486   6.042493   5.561648  
      LDA-Unif    3.100   7.781870  31.331217  
      LDA-Pop     3.251   8.209632  31.252544

In [68]:
print(fit_merge.to_latex(formatters=[
    ((lambda v: '{:.2f}%'.format(v * 100))
     if t == 'Loss'
     else (lambda v: '{:.3f}'.format(v)))
for (m, t) in fit_merge.columns], multirow=True, multicolumn_format='c'))

\begin{tabular}{llrrrrrrrrr}
\toprule
      &         & \multicolumn{2}{c}{I-I Sim} & \multicolumn{2}{c}{Item Pop} & \multicolumn{2}{c}{U-U Sim} & \multicolumn{2}{c}{User Act} &      Avg \\
      &         &     KLD &     Loss &      KLD &     Loss &     KLD &     Loss &      KLD &     Loss &     Loss \\
Data & Model &         &          &          &          &         &          &          &          &          \\
\midrule
\multirow{4}{*}{ML1M} & IBP-Unif &   0.170 & 8400.00\% &    0.973 &  112.45\% &   0.227 & 3683.33\% &    0.373 &   42.37\% & 3059.54\% \\
      & IBP-Pop &   0.200 & 9900.00\% &    1.793 &  291.48\% &   0.170 & 2733.33\% &    0.388 &   48.09\% & 3243.23\% \\
      & LDA-Unif &   0.137 & 6750.00\% &    0.737 &   60.92\% &   0.082 & 1266.67\% &    0.669 &  155.34\% & 2058.23\% \\
      & LDA-Pop &   0.050 & 2400.00\% &    2.063 &  350.44\% &   0.291 & 4750.00\% &    1.084 &  313.74\% & 1953.54\% \\
\cline{1-11}
\multirow{4}{*}{AZM5} & IBP-Unif &   0.024 &  500.00\% & 